In [1]:
import os
import time
import decord
from omegaconf import OmegaConf

current_dir = os.getcwd()
target_dir_name = 'MimicMotion'

target_dir = os.path.join(current_dir, target_dir_name)

if not current_dir.endswith(target_dir_name):
    if os.path.exists(target_dir):
        os.chdir(target_dir)
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print(f"Target directory {target_dir} does not exist.")
else:
    print(f"Already in the target directory: {current_dir}")

In [2]:
import concurrent.futures
import random
import math
from pathlib import Path
import numpy as np
import argparse

import subprocess
import cv2
import json
from PIL import Image
from controlnet_aux.util import HWC3, resize_image

from mimicmotion.dwpose.util import draw_bodypose, draw_handpose, draw_facepose
from torchvision.datasets.folder import pil_loader
from torchvision.transforms.functional import pil_to_tensor, resize, center_crop
from torchvision.transforms.functional import to_pil_image

from mimicmotion.utils.utils import get_fps, save_videos_from_pil, read_frames

/root/miniconda3/lib/python3.10/site-packages/skimage/morphology/binary.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy import ndimage as ndi


In [3]:
args = argparse.Namespace(
    video_root="../datas/TikTok_Crop_Videos", 
    save_dir=None, 
    j=1
)

print(args.video_root)
if not os.path.exists(args.video_root):
    raise ValueError(f"The specified video root path does not exist: {args.video_root}")

../datas/TikTok_Crop_Videos


In [4]:
num_workers = args.j

if args.save_dir is None:
    save_dir = args.video_root + "_dwpose"
else:
    save_dir = args.save_dir
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES", "0")
gpu_ids = [int(id) for id in range(len(cuda_visible_devices.split(",")))]
print(f"avaliable gpu ids: {gpu_ids}")

avaliable gpu ids: [0]


In [5]:
# collect all video_folder paths
video_mp4_paths = set()
for root, dirs, files in os.walk(args.video_root):
    for name in files:
        if name.endswith(".mp4"):
            video_mp4_paths.add(os.path.join(root, name))
video_mp4_paths = list(video_mp4_paths)
random.shuffle(video_mp4_paths)

# crop video 【裁剪视频】

In [6]:
def cropp_single_video(video_path, root_dir, save_dir, detect_resolution=512, image_resolution=512, output_type="pil"):
    relative_path = os.path.relpath(video_path, root_dir)
    
    out_path = os.path.join(save_dir, relative_path)
    print('relative_path, video_path, root_dir', relative_path, video_path, root_dir)
    if os.path.exists(out_path):
        return

    output_dir = Path(os.path.dirname(os.path.join(save_dir, relative_path)))
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        
    fps = get_fps(video_path)
    frames = read_frames(video_path)
    kps_results = []
    
    for i, frame_pil in enumerate(frames):
        print(f"Processing frame {i+1}", end='\r')
        
        input_image = cv2.cvtColor(
            np.array(frame_pil, dtype=np.uint8), cv2.COLOR_RGB2BGR
        )
        input_image = HWC3(input_image)
        input_image = resize_image(input_image, detect_resolution)
        H, W, C = input_image.shape
        
        input_image = cv2.resize(
            input_image, (W, H), interpolation=cv2.INTER_LINEAR
        )
        if output_type == "pil":
            input_image = Image.fromarray(input_image)
        
        kps_results.append(input_image)
        
    save_videos_from_pil(kps_results, out_path, fps=fps)

def cropp_batch_videos(video_list, root_dir, save_dir):
    for i, video_path in enumerate(video_list):
        print(f"Processing video {i + 1}/{len(video_list)}")
        cropp_single_video(video_path, root_dir, save_dir)

In [7]:
crop_video_root ="../datas/TikTok_Crop_Videos"
crop_save_dir = "../datas/TikTok_Crop_Videos2"
cropp_batch_videos(video_mp4_paths, root_dir=crop_video_root, save_dir=crop_save_dir)

Processing video 1/26
relative_path, video_path, root_dir seq_00231_00240/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00231_00240/YouTube.mp4 ../datas/TikTok_Crop_Videos
Processing video 2/26
relative_path, video_path, root_dir seq_00029_00049/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00029_00049/YouTube.mp4 ../datas/TikTok_Crop_Videos
Processing video 3/26
relative_path, video_path, root_dir seq_00289_00307/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00289_00307/YouTube.mp4 ../datas/TikTok_Crop_Videos
Processing video 4/26
relative_path, video_path, root_dir seq_00167_00188/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00167_00188/YouTube.mp4 ../datas/TikTok_Crop_Videos
Processing video 5/26
relative_path, video_path, root_dir seq_00308_00328/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00308_00328/YouTube.mp4 ../datas/TikTok_Crop_Videos
Processing video 6/26
relative_path, video_path, root_dir seq_00058_00077/YouTube.mp4 ../datas/TikTok_Crop_Videos/seq_00058_00077/YouTube.mp4 ../datas

In [8]:
import subprocess
import os

def crop_single_video_ffmpeg(video_path, root_dir, save_dir, start_time, end_time):
    # 计算相对路径
    relative_path = os.path.relpath(video_path, root_dir)
    
    # 确定输出路径，并保留原始文件名
    out_path = os.path.join(save_dir, relative_path)
    output_dir = os.path.dirname(out_path)
    
    # # 如果输出文件已经存在，则跳过
    # if os.path.exists(out_path):
    #     return

    # 创建输出目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    print('relative_path, video_path, root_dir:', relative_path, video_path, root_dir)
    print('output_dir:', output_dir)

    # 使用 ffmpeg 进行视频裁剪
    command = [
        'ffmpeg',
        '-y', # 覆盖已有文件
        '-i', video_path,
        '-ss', str(start_time),
        '-to', str(end_time),
        '-c:v', 'libx264',
        '-c:a', 'aac',
        '-strict', 'experimental',
        out_path
    ]
    
    subprocess.run(command, check=True)
    print(f"Cropped video saved to {out_path}")

def crop_batch_videos_ffmpeg(video_list, root_dir, save_dir, start_time, end_time):
    for i, video_path in enumerate(video_list):
        print(f"Processing video {i + 1}/{len(video_list)}")
        crop_single_video_ffmpeg(video_path, root_dir, save_dir, start_time, end_time)


In [9]:
start_time = 0 
end_time = 9
crop_video_root ="../datas/TikTok_Crop_Videos"
crop_save_dir = "../datas/TikTok_Crop_Videos2"
crop_batch_videos_ffmpeg(crop_video_root, root_dir=crop_video_root, save_dir=crop_save_dir, start_time=start_time, end_time=end_time)

Processing video 1/26
relative_path, video_path, root_dir: seq_00282_00288/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos/seq_00282_00288/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos
output_dir: ../autodl-tmp/TikTok_Crop_Videos/seq_00282_00288


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Cropped video saved to ../autodl-tmp/TikTok_Crop_Videos/seq_00282_00288/YouTube.mp4
Processing video 2/26
relative_path, video_path, root_dir: seq_00241_00244/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos/seq_00241_00244/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos
output_dir: ../autodl-tmp/TikTok_Crop_Videos/seq_00241_00244


[libx264 @ 0x55da0e091fc0] using SAR=1/1
[libx264 @ 0x55da0e091fc0] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x55da0e091fc0] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x55da0e091fc0] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=34 lookahead_threads=5 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '../autodl-tmp/TikTok_Crop_Videos/seq_00241_00244/YouTube.mp4':
  Metadata:
    

Cropped video saved to ../autodl-tmp/TikTok_Crop_Videos/seq_00241_00244/YouTube.mp4
Processing video 3/26
relative_path, video_path, root_dir: seq_00245_00248/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos/seq_00245_00248/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos
output_dir: ../autodl-tmp/TikTok_Crop_Videos/seq_00245_00248


...

Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
Press [q] to stop, [?] for help
[libx264 @ 0x55a831c47500] using SAR=1/1
[libx264 @ 0x55a831c47500] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x55a831c47500] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x55a831c47500] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=34 lookahead_threads=5 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:

Cropped video saved to ../autodl-tmp/TikTok_Crop_Videos/seq_00010_00028/YouTube.mp4
Processing video 26/26
relative_path, video_path, root_dir: seq_00167_00188/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos/seq_00167_00188/YouTube.mp4 ../autodl-tmp/TikTok_Raw_Videos/TikTok_Raw_Videos
output_dir: ../autodl-tmp/TikTok_Crop_Videos/seq_00167_00188


[libx264 @ 0x55819ad3ae00] using SAR=1/1
[libx264 @ 0x55819ad3ae00] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2 AVX512
[libx264 @ 0x55819ad3ae00] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x55819ad3ae00] 264 - core 163 r3060 5db6aa6 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=34 lookahead_threads=5 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '../autodl-tmp/TikTok_Crop_Videos/seq_00167_00188/YouTube.mp4':
  Metadata:
    

Cropped video saved to ../autodl-tmp/TikTok_Crop_Videos/seq_00167_00188/YouTube.mp4


frame=  270 fps= 87 q=-1.0 Lsize=    2092kB time=00:00:08.90 bitrate=1925.4kbits/s speed=2.87x    
video:2088kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.196179%
[libx264 @ 0x55819ad3ae00] frame I:2     Avg QP:16.41  size: 38302
[libx264 @ 0x55819ad3ae00] frame P:70    Avg QP:18.49  size: 14831
[libx264 @ 0x55819ad3ae00] frame B:198   Avg QP:22.00  size:  5163
[libx264 @ 0x55819ad3ae00] consecutive B-frames:  1.5%  2.2%  0.0% 96.3%
[libx264 @ 0x55819ad3ae00] mb I  I16..4: 25.0% 67.9%  7.0%
[libx264 @ 0x55819ad3ae00] mb P  I16..4:  6.7% 19.4%  0.9%  P16..4: 11.4%  3.4%  1.4%  0.0%  0.0%    skip:56.8%
[libx264 @ 0x55819ad3ae00] mb B  I16..4:  0.8%  1.6%  0.1%  B16..8: 17.0%  2.1%  0.4%  direct: 1.3%  skip:76.7%  L0:45.8% L1:50.3% BI: 3.9%
[libx264 @ 0x55819ad3ae00] 8x8 transform intra:70.0% inter:84.7%
[libx264 @ 0x55819ad3ae00] coded y,uvDC,uvAC intra: 21.7% 29.2% 2.4% inter: 3.0% 3.9% 0.0%
[libx264 @ 0x55819ad3ae00] i16 v,h,dc,p: 46% 22% 10% 23%
[li

# extract_dwpose_from_vid 【提取姿势】

In [10]:
from mimicmotion.dwpose.dwpose_detector import dwpose_detector as dwprocessor

2024-08-14 17:34:46.713184848 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-08-14 17:34:46.714263472 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [11]:
def draw_pose(pose, H, W):
    bodies = pose["bodies"]
    
    faces = pose["faces"]
    faces_score = pose["faces_score"]
    hands = pose["hands"]
    hands_score = pose["hands_score"]
    
    candidate = bodies["candidate"]
    subset = bodies["subset"]
    score = bodies["score"]
    canvas = np.zeros(shape=(H, W, 3), dtype=np.uint8)
    
    canvas = draw_bodypose(canvas, candidate, subset, score)
    
    canvas = draw_handpose(canvas, hands, hands_score)

    canvas = draw_facepose(canvas, faces, faces_score)

    return canvas

In [16]:
def process_single_video(video_path, detector, root_dir, save_dir, detect_resolution=512, image_resolution=512, output_type="pil"):
    relative_path = os.path.relpath(video_path, root_dir)
    
    out_path = os.path.join(save_dir, relative_path)
    print('relative_path, video_path, root_dir', relative_path, video_path, root_dir)
    if os.path.exists(out_path):
        return

    output_dir = Path(os.path.dirname(os.path.join(save_dir, relative_path)))
    if not output_dir.exists():
        output_dir.mkdir(parents=True, exist_ok=True)
        
    fps = get_fps(video_path)
    frames = read_frames(video_path)
    kps_results = []
    
    for i, frame_pil in enumerate(frames):
        print(f"Processing frame {i+1}", end='\r')
        
        input_image = cv2.cvtColor(
            np.array(frame_pil, dtype=np.uint8), cv2.COLOR_RGB2BGR
        )
        input_image = HWC3(input_image)
        input_image = resize_image(input_image, detect_resolution)
        H, W, C = input_image.shape
        
        pose = detector(input_image)
        
        detected_map = draw_pose(pose, H, W)
        detected_map = HWC3(detected_map)
        
        img = resize_image(input_image, image_resolution)
        H, W, C = img.shape

        detected_map = cv2.resize(
            detected_map, (W, H), interpolation=cv2.INTER_LINEAR
        )
        if output_type == "pil":
            detected_map = Image.fromarray(detected_map)
        
        kps_results.append(detected_map)
        
    save_videos_from_pil(kps_results, out_path, fps=fps)

def process_batch_videos(video_list, detector, root_dir, save_dir):
    for i, video_path in enumerate(video_list):
        print(f"Process {i}/{len(video_list)} video")
        process_single_video(video_path, detector, root_dir, save_dir)

In [17]:
pose_root_dir = "../autodl-tmp/TikTok_Crop_Videos"
pose_save_dir = pose_root_dir + "_dwpose"

pose_mp4_paths = set()
for root, dirs, files in os.walk(pose_root_dir):
    for name in files:
        if name.endswith(".mp4"):
            pose_mp4_paths.add(os.path.join(root, name))
pose_mp4_paths = list(pose_mp4_paths)

In [18]:
detector = dwprocessor
# detector = DWposeDetector()
process_batch_videos(pose_mp4_paths, detector, root_dir=pose_root_dir, save_dir=pose_save_dir)

Process 0/26 video
relative_path, video_path, root_dir seq_00198_00203/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos/seq_00198_00203/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos
Process 1/26 video70
relative_path, video_path, root_dir seq_00245_00248/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos/seq_00245_00248/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos
Process 2/26 video70
relative_path, video_path, root_dir seq_00058_00077/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos/seq_00058_00077/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos
Process 3/26 video70
relative_path, video_path, root_dir seq_00204_00230/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos/seq_00204_00230/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos
Process 4/26 video70
relative_path, video_path, root_dir seq_00010_00028/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos/seq_00010_00028/YouTube.mp4 ../autodl-tmp/TikTok_Crop_Videos
Process 5/26 video70
relative_path, video_path, root_dir seq_00167_00188/YouTube.mp4 ../autodl-tmp/TikTok_C

# extract_meta_info【制作路径data】

In [9]:
args = argparse.Namespace(
    root_path="../datas/TikTok_Crop_Videos", 
    dataset_name="TikTok", 
    meta_info_name=None
)
if args.meta_info_name is None:
    args.meta_info_name = args.dataset_name

In [10]:
pose_dir = args.root_path + "_dwpose"
hand_dir = args.root_path + "_dwhand"
hand_mask_dir = args.root_path + "_dwmask"

# collect all video_folder paths
video_mp4_paths = set()
for root, dirs, files in os.walk(args.root_path):
    for name in files:
        if name.endswith(".mp4"):
            video_mp4_paths.add(os.path.join(root, name))
video_mp4_paths = list(video_mp4_paths)

In [11]:
meta_infos = []
for video_mp4_path in video_mp4_paths:
    relative_video_name = os.path.relpath(video_mp4_path, args.root_path)
    kps_path = os.path.join(pose_dir, relative_video_name)
    hand_path = os.path.join(hand_dir, relative_video_name)
    hand_mask_path = os.path.join(hand_mask_dir, relative_video_name)
    meta_infos.append({"video_path": video_mp4_path, "kps_path": kps_path, "hand_path": hand_path, "hand_mask_path": hand_mask_path})

json.dump(meta_infos, open(f"dataset/data/{args.meta_info_name}_meta.json", "w"))